<a href="https://colab.research.google.com/github/kcwanglucky/filetransfer/blob/master/app_comment_generate_reply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Provide "/content/20190508起APP評論.xlsx" or whatever app comment file with similar format  
* Auto-generate a new column "自動回覆內容" on the original xlsx file and then save at "autoresponse.xlsx"

In [0]:
!pip install transformers

Idea:
1.   13 reponses with > 5 counts automatically form groups. Try to put the other 129 types into the aforementioned 13 groups if certain conditions meet (consine similarity of Bert embeddings, etc.). If not, put into a separate group.

    > Result:
        *   Directly assigned to the group with highest cossim: 60ep - train acc~ 99%, test acc~ 45%
        *   If max consim < 98.5%, create a separate group: 60ep - train acc~ 98.4%, test acc~ 44%
        *   dropna the "index" and "question" columns:  
        1) 15ep - train acc~ 98%, test acc~ 73% (ep15_eg5_ml30)  
        2) 35ep - train acc~ 99%, test acc~ 65% (param: ep35_eg5_ml50)  
        3) model_ep20_eg0_ml40 - train acc~ 99%, test acc~ 63%  
        4) model_ep25_eg5_ml30 - train acc~ 99%, test acc~ 72.3%  
        5) model_ep25_eg0_ml30 - train acc~ 99%, test acc~ 66%  
        6) model_ep25_eg0_ml50 - train acc~ 98%, test acc~ 68%  
        7) model_ep25_eg5_ml30 - train acc~ 99%, test acc~ 71.5%  
        





In [0]:
#from BertSequenceClassification_streamline.bert_downstream_classification import *
#from BertSequenceClassification_streamline.app_comment import *

In [0]:
from app_utility import *
from bert_classification import *

In [0]:
"""set the global variables"""
epoch = 20
batch_size = 64
min_each_group = 0                      # 每組至少要有幾筆才被放入訓練集
max_length = 150                          # 若app comment評論超過此長度則刪除
model_output = 'model_ep20_eg0_ml150'
pretrained_bert_name = "bert-base-chinese"
#model_start = "app_all1"

In [0]:
path = "20190508起APP評論.xlsx"
df = pd.read_excel(path)
df = preprocess_app_comment(df, verbose = False)
df = filter_toofew_toolong(df, min_each_group, max_length)
df = append_question_title(df)          # 將問題的標題 append 到問題內文
df = df.loc[:, ["index", "question"]]   # 只取 'index' 跟 'question' coluumn
#df.to_csv("app_comment" + "_me" + str(args.min_each_group) + "_ml" + str(args.maxlength) + ".csv")

In [0]:
# Investigate all data
app_data = AppCommentData(df, mode = "all")
#app_data.plot_pie(fontprop)
_ = app_data.get_index_dist(verbose = True)

all data 各類分布: 

稱讚              數量: 648
未知問題            數量: 556
抱怨              數量: 244
操作問題            數量: 190
系統問題-Keypasco   數量: 115
建議-APP          數量: 96
建議-信用卡          數量: 89
系統問題-bug        數量: 58
顧客疑問            數量: 47
建議-台幣           數量: 37
第三方問題           數量: 20
建議-已規劃          數量: 5
建議-財富           數量: 3
建議-外幣           數量: 2


In [0]:
num_labels = app_data.get_num_index()
index2label = app_data.get_index2label()
label2index = app_data.get_label2index()

In [0]:
# Sampling 1: Random split
#df_train, df_val, df_test = train_val_test_split(df)

# Sampling 2: Sampling within each group
df_train, df_val, df_test = output_split(df, 0.75)

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_bert_name)


訓練樣本數： 1583
validation樣本數： 222
預測樣本數： 221


In [0]:
# investigate train set
app_train = AppCommentData(df_train, "train", tokenizer, 64)
app_train.reindex(label2index)     # 訓練前需要 reindex，不然若 output label 不是連續的話則預測結果會有問題
#app_train.plot_pie(fontprop)

In [0]:
# investigate test set
app_val = AppCommentData(df_val, "val", tokenizer, 64)
app_val.reindex(label2index)
app_test = AppCommentData(df_test, "test", tokenizer, 64)
app_test.reindex(label2index)
#app_test.plot_pie(fontprop)

In [0]:
trainloader = app_train.get_dataloader()
valloader = app_val.get_dataloader()
testloader = app_test.get_dataloader()

In [0]:
model = Model(df, pretrained_bert_name)
_ = model.train(trainloader, valloader, epoch)

device: cuda:0

======== Epoch 1 / 20 ========
Training...
loss: 55.668, acc: 0.504

Running Validation...
  Accuracy: 0.41

======== Epoch 2 / 20 ========
Training...
loss: 43.955, acc: 0.539

Running Validation...
  Accuracy: 0.43

======== Epoch 3 / 20 ========
Training...
loss: 36.707, acc: 0.606

Running Validation...
  Accuracy: 0.47

======== Epoch 4 / 20 ========
Training...
loss: 31.172, acc: 0.669

Running Validation...
  Accuracy: 0.56

======== Epoch 5 / 20 ========
Training...
loss: 26.288, acc: 0.742

Running Validation...
  Accuracy: 0.61

======== Epoch 6 / 20 ========
Training...
loss: 22.315, acc: 0.776

Running Validation...
  Accuracy: 0.61

======== Epoch 7 / 20 ========
Training...
loss: 19.414, acc: 0.829

Running Validation...
  Accuracy: 0.59

======== Epoch 8 / 20 ========
Training...
loss: 17.155, acc: 0.867

Running Validation...
  Accuracy: 0.64

======== Epoch 9 / 20 ========
Training...
loss: 15.048, acc: 0.876

Running Validation...
  Accuracy: 0.63

===

In [0]:
model.save_model(model_output)

Saving model to model_ep20_eg0_ml150


In [0]:
predictions = get_predictions(model.model, testloader).detach().cpu().numpy()

In [0]:
# testing
#predictions = get_predictions(model, testloader).detach().cpu().numpy()
#predictions = model.predict(testloader)

true_label = app_test.df_reindex["index"]
class_matrix, false_group = get_confusion_matrix(true_label, predictions, num_labels)
print_acc(class_matrix)

# label_list = [index2label[key] for key in index2label]
# label2index = {val:key for key, val in index2label.items()}
# summary = pd.DataFrame(class_matrix, dtype = int, columns=label_list, index = label_list)
# print("\n測試集 confusion matrix:")
# summary

Accuracy: 59.276018099547514%


In [0]:
class_matrix

array([[44.,  5.,  5.,  8.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,
         5.],
       [ 2., 41.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.],
       [ 7.,  1.,  8.,  4.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 4.,  1.,  4., 17.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.],
       [ 0.,  0.,  0.,  0.,  2.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.],
       [ 2.,  0.,  3.,  3.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 3.,  1.,  3.,  0.,  0.,  0.,  1.,  0.,  0.,  3.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0., 

### 自動回覆模板

In [0]:
label2response = {'建議-APP': '親愛的顧客您好\n感謝您的支持與愛護，本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~', 
                  '建議-信用卡': '親愛的顧客您好\n感謝您的支持與愛護，本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~',
                  '建議-台幣': '親愛的顧客您好\n感謝您的支持與愛護，本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~', 
                  '抱怨': '親愛的顧客您好\n造成您的不便，敬請見諒。本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~',
                  '操作問題': '操作問題',
                  '未知問題': '親愛的顧客您好\n感謝您的回饋，為了進一步確認您的問題，以便把您的意見回饋給相關單位，請您使用玉山銀行訪客留言板(https://www.esunbank.com.tw/bank/about/services/customer/message-board)提供您的個人資訊及行動裝置資訊(如手機型號、系統版本、APP版號)或使用智能客服(https://robot.esunbank.com.tw/index.php?eservice=footer&qaCategory=general)，我們將為您服務，謝謝您~。',
                  '稱讚': '親愛的顧客您好\n您的支持與愛護是玉山服務更精進的動力。謝謝您~',
                  '第三方問題': '第三方問題',
                  '系統問題-Keypasco': '親愛的顧客您好\n感謝您的回饋，請您嘗試依照下列步驟重新操作。\n步驟(1)：進入行動銀行APP→選擇側邊選單→ 「設定」→「行動裝置綁定管理」中取消綁定所有設備。\n步驟(2)： 開啟 Google Play， 搜尋「玉山行動銀行」APP進行解除安裝，並關機。\n步驟(3)： 重新開機下載行動銀行，再操作裝置綁定及圖形密碼登入流程。\n造成您的不便，敬請見諒。\n若仍無法改善您的問題，為了進一步確認您的問題，請您使用玉山銀行訪客留言板(https://www.esunbank.com.tw/bank/about/services/customer/message-board)，提供您的個人資訊及行動裝置資訊(如手機型號、系統版本、APP版號)，謝謝您~',
                  '系統問題-bug': '親愛的顧客您好\n感謝您的回饋，目前已盡速修復中，造成您的不便，敬請見諒。\n如有任何其他問題，建議您可使用智能客服(https://robot.esunbank.com.tw/index.php?eservice=footer&qaCategory=general)，我們將為您服務，謝謝您~',
                  '顧客疑問': '顧客疑問',
                  '建議-外幣': '親愛的顧客您好\n感謝您的回饋。本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~',
                  '建議-財富': '親愛的顧客您好\n感謝您的回饋。本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~',
                  '建議-已規劃': '親愛的顧客您好\n感謝您的支持與愛護，本行將持續改善與精進，我們會把您的意見回饋給相關單位，您寶貴的意見都是讓玉山服務更精進的動力。謝謝您~',
                  }

In [0]:
index2label = {0: '未知問題',
                1: '稱讚',
                2: '操作問題',
                3: '抱怨',
                4: '建議-台幣',
                5: '建議-已規劃',
                6: '顧客疑問',
                7: '建議-信用卡',
                8: '第三方問題',
                9: '建議-APP',
                10: '系統問題-bug',
                11: '建議-財富',
                12: '建議-外幣',
                13: '系統問題-Keypasco'}

In [0]:
def get_response(df, model):
    """讀取 app 評論，用 model 進行預測後，增加一欄為自動回覆的內容
    Args:
        df: app 評論資料，需有 "question" 這一欄
        model: 用來進行意圖預測的 model
    Returns:
        response: A list of string with designated response
    """
    app_data = AppCommentData(df, "test", tokenizer, 64)
    dataloader = app_data.get_dataloader()
    predictions = get_predictions(model, dataloader).detach().cpu().numpy()
    
    pred_label = [index2label[i] for i in predictions]
    response = [label2response[label] for label in pred_label]
    return response

In [0]:
def generate_response(inpath, outpath, model):
    df = pd.read_excel(inpath)
    df = preprocess_app_comment(df, verbose = False)
    df = filter_toofew_toolong(df, min_each_group, max_length)
    df = append_question_title(df)          # 將問題的標題 append 到問題內文
    df = df.loc[:, ["index", "question"]]   # 只取 'index' 跟 'question' coluumn
    
    response = get_response(df, model)
    
    df_autoreply = preprocess_app_comment(inpath)
    df_autoreply = filter_toofew_toolong(df_autoreply, args.min_each_group, args.maxlength)
    df_autoreply = df_autoreply.iloc[:, :-3]

    df_autoreply['自動回覆內容'] = response
    df_autoreply.to_excel(outpath)

In [0]:
inpath = path
model_path = "model_ep20_eg0_ml150"
model = torch.load(model_path)
outpath = "autoresponse.xlsx"
generate_response(inpath, outpath, model)

IsADirectoryError: ignored

### Presentation

In [0]:
# 個別檢查
# labels: ('未知問題','操作問題','稱讚','建議-台幣','抱怨','建議-信用卡','建議-APP','顧客疑問','系統問題-bug','第三方問題','系統問題-Keypasco')
i = label2index['未知問題']
print_summary_i(app_test.df, i, class_matrix, index2label, false_group, predictions, firstK = 5)


類別: 未知問題 	 測試集筆數: 64
被分類器正確分類的機率: 76.5625 %
最常被分錯的組別: 抱怨  筆數: 6.0

轉帳明細頁面壞了嗎明明有轉帳了 
但交易明細頁面都沒顯示出金額耶！
不知道哪時能修好 不然很麻煩😥
	被分成: 抱怨
存摺備註
	被分成: 建議-台幣
以前有用過玉山銀行的信用卡，但點書不知有沒有剩餘價值
	被分成: 顧客疑問
台幣活存、台幣定存、外幣活存、外幣定存加總起來都跟首頁圓圈內的數字不吻合。請開發者修正此問題，謝謝。
	被分成: 顧客疑問
信用卡分期還要跟銀行要授權碼??其他銀行都不用就玉山最多此一舉
	被分成: 操作問題


### save/read model from gdrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r model_ep20_eg0_ml150/ "drive/My Drive/ESUN"

In [0]:
!cp -r 待檢查_除只有組織.xlsx "drive/My Drive/"

In [0]:
!cp -r /content/model_ep20_eg0_ml100 "drive/My Drive/ESUN/0317_app_comment"

In [0]:
!cp -r "drive/My Drive/ESUN/0317_app_comment/model_ep20_eg0_ml100" .

In [0]:
model_dir = "model_ep20_eg0_ml100"
model = BertForSequenceClassification.from_pretrained(model_dir)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained(model_dir)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element